## Cython
Material basado en:
https://github.com/aspp-latam/2023-mexico-profiling/blob/main/cython.ipynb

Para instalar Cython utilizamos la vieja confiable:

_pip install Cython_

En el caso de Windows es necesario tener el compilador de C que puede ser Visual Studio

### 1. Python puro

In [17]:
def f(x):
    return x**4-3*x

def integrate_f(func, a, b, n):
    dx = (b-a)/n
    s = 0.0
    for i in range(n):
        s += func(a+(i+0.5)*dx)*dx
    return s

In [18]:
integrate_f(f, -10, 10, 1_000_000)

39999.999999868494

In [19]:
%timeit integrate_f(f, -10, 10, 1_000_000)

290 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2. Compilando Python en Cython

In [20]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [21]:
%%cython

def f2(x):
    return x**4-3*x

def integrate_f2(func, a, b, n):
    dx = (b-a)/n
    s = 0.0
    for i in range(n):
        s += func(a+(i+0.5)*dx)*dx
    return s

Content of stdout:
_cython_magic_6fbfbd07070c621a30aacd3c70338b70bc49c349.c
   Creando biblioteca C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_6fbfbd07070c621a30aacd3c70338b70bc49c349.cp311-win_amd64.lib y objeto C:\Users\dacr9\.ipython\cython\Users\dacr9\.ipython\cython\_cython_magic_6fbfbd07070c621a30aacd3c70338b70bc49c349.cp311-win_amd64.exp
Generando c¢digo
Generaci¢n de c¢digo finalizada

In [22]:
integrate_f2(f2, -10, 10, 1_000_000)

39999.999999868494

In [8]:
%timeit integrate_f2(f2, -10, 10, 1_000_000)

229 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3. Especialización de variables

In [9]:
%%cython

def f3(double x):
    return x**4-3*x

def integrate_f3(func, double a, double b, int n):
    dx = (b-a)/n
    s = 0.0
    for i in range(n):
        s += func(a+(i+0.5)*dx)*dx
    return s

In [10]:
%timeit integrate_f3(f3, -10, 10, 1_000_000)

136 ms ± 3.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 4. Variables internas

In [11]:
%%cython

def f4(double x):
    return x**4-3*x

def integrate_f4(func, double a, double b, int n):
    cdef:
        double dx = (b-a)/n
        double s = 0.0
        int i

    for i in range(n):
        s += func(a+(i+0.5)*dx)*dx
    return s

In [12]:
%timeit integrate_f4(f4, -10, 10, 1_000_000)

83.5 ms ± 584 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 5. Especialización de funciones

In [13]:
%%cython

cdef double f5(double x):
    return x**4-3*x

def integrate_f5(double a, double b, int n):
    cdef:
        double dx = (b-a)/n
        double s = 0.0
        int i

    for i in range(n):
        s += f5(a+(i+0.5)*dx)*dx
    return s

In [14]:
%timeit integrate_f5(-10, 10, 1_000_000)

28.7 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 6. Reescritura de operaciones

In [15]:
%%cython

cdef double f6(double x):
    return (x*x*x-3)*x

def integrate_f6(double a, double b, int n):
    cdef:
        double dx = (b-a)/n
        double s = 0.0
        int i

    for i in range(n):
        s += f6(a+(i+0.5)*dx)*dx
    return s

In [16]:
%timeit integrate_f6(-10, 10, 1_000_000)

1.45 ms ± 8.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


| Estrategia |Tiempo |
|----------|----------|
| Python puro          | 290 ms   | 
| Compilando Python    | 229 ms   | 
| Especialización de variables | 136 ms   | 
| Variables internas   | 83.5 ms |
| Especialización de funciones | 28.7 |
| Reescritura de operaciones   | 1.45 ms |   